## Extraction des données

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd 
from time import sleep
import time
import selenium
import pandas as pd
from selenium import webdriver  
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import json
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import psycopg2
from sqlalchemy import create_engine

In [ ]:
def scrolling_offre():
    SCROLL_PAUSE_TIME = 1.8
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    return 

In [ ]:
def get_descriptions(driver):

    scrolling_offre()
    src_description =driver.page_source
    soupe1 =BeautifulSoup(src_description, 'lxml')
    try:

        titre1_element = soupe1.find('h1',class_="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title")
        titre2_element=titre1_element.find_all('span')[0]
        titre_element = titre2_element.text.strip()
        
        
    except:
        titre_element="none"

    try:
        details_elements = soupe1.find('div',class_="css-tvvxwd ecydgvn1")
        type_contrat_element=details_elements.text.strip()

        
    except:
        type_contrat_element="none"

    try:
        div_element = soupe1.find('div', id='jobDescriptionText')
    
        if div_element:
            paragraphs = div_element.find_all('p') 
            paragraphs1 = div_element.find_all('li')  

            if paragraphs or paragraphs1:
                content = ""  

                for paragraph in paragraphs:
                    content += paragraph.get_text(strip=True)
                for paragraph1 in paragraphs1:
                    content += paragraph1.get_text(strip=True) 
            
            else:
                content = div_element.get_text(strip=True) 
                
    

    except:
        content="none"
        
    try:
        nom_societe0 = soupe1.find('div',class_="css-1cjkto6 eu4oa1w0")
        nom_societe1=nom_societe0.find_all('a')[0]
        nom_societe = nom_societe1.text.strip()
    except:
        nom_societe="none"
        
    try:
        ville0 = soupe1.find('div',class_="css-6z8o9s eu4oa1w0")
        ville1=ville0.find_all('div')[0]
        ville = ville1.text.strip()
    except:
        ville="none"
    return titre_element,type_contrat_element,content,nom_societe,ville

In [ ]:
def get_details(href_list, link, driver):
    titre =[]
    type_contract =[]
    description_offre =[]
    nom_societe=[]
    ville=[]

    main_window = driver.current_window_handle  
    for lien in href_list:
        url_complet = link + lien
        driver.execute_script("window.open('" + url_complet + "');")
        new_window = driver.window_handles[-1] 
        
        driver.switch_to.window(new_window)
        sleep(3)
        result=get_descriptions(driver)
        titre.append(result[0])
        type_contract.append(result[1])
        description_offre.append(result[2])
        nom_societe.append(result[3])
        ville.append(result[4])
        driver.close()  
        driver.switch_to.window(main_window)
    return titre,type_contract,description_offre,nom_societe,ville

In [ ]:
link="https://ca.indeed.com"
driver = webdriver.Chrome("C:\\Users\\gaith\\OneDrive\\Bureau\\projects\\chromedriver.exe")
sleep(1)
url="entrer le lien de indeed"
driver.get(url)
href_list = []
dates2=[]
for i in range(1):
    scrolling_offre()
    sleep(1)
    src =driver.page_source
    soupe =BeautifulSoup(src, 'lxml')
    dates1 = soupe.find_all('span', {'class': 'date'})
    containers = soupe.find_all('a', {'class': 'jcs-JobTitle css-jspxzf eu4oa1w0'})
    for date0 in dates1:
        dates2.append(date0.text.strip())
    for container in containers:
        href = container.get('href')
        href_list.append(href)
data=get_details(href_list, link, driver)


In [ ]:
statut=[]
categorie=[]
description=[]
dates=[]
societe=[]
ville=[]
statut.extend(data[0])
categorie.extend(data[1])
description.extend(data[2])
societe.extend(data[3])
ville.extend(data[4])
dates.extend(dates2)

offresd_emploi = pd.DataFrame(
{'statut': statut,
'type_poste': categorie,
'description': description,       
"societe":societe,
 "ville":ville,
 "dates":dates
            })
offresd_emploi= offresd_emploi.drop_duplicates()

## Traitement de données

In [ ]:
nan_counts = offresd_emploi.isna().sum()
nan_counts 

In [ ]:
def extract_number(text):
    match = re.search(r'\d+', str(text))
    if match:
        return int(match.group())
    else:
        return None

offresd_emploi['dates'] = offresd_emploi['dates'].apply(extract_number).astype('Int64')

def replace(value):
    if value in range(1, 8):
        return '4ème semaine'
    elif value in range(8, 15):
        return '3ème semaine'
    elif value in range(15, 22):
        return '2ème semaine'
    elif value in range(22, 30):
        return '1ère semaine'
    else:
        return "il y a un mois"

offresd_emploi['dates'] = offresd_emploi['dates'].apply(replace)

In [ ]:
offresd_emploi['ville'] = offresd_emploi['ville'].apply(lambda x: re.sub(r'[\d()]+', '', x))

In [ ]:
categorie = input("Entrez la categorie des offres d'emploi scrappe : ")
sous_categorie  = input("Entrez la sous categorie des offres d'emploi scrappe : ")
offresd_emploi['categorie'] = [categorie] * len(offresd_emploi)
offresd_emploi['sous_categorie'] = [sous_categorie] * len(offresd_emploi)

In [ ]:
colonnes_specifiques = ['categorie', 'sous_categorie',"type_poste","societe","ville","dates","statut"]
df_offresd_emploi = offresd_emploi.loc[:, colonnes_specifiques]

In [ ]:
df_offresd_emploi

## Loading des données

In [ ]:
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    database="offre_emploi",
    user="postgres",
    password="########"
)

engine = create_engine('postgresql+psycopg2://', creator=lambda: conn)
df_ville.to_sql('ville', con=engine, if_exists='append', index=False)
conn.close()
